# Innomatics Research Labs | Advanced GenAI Internship | Entrance Test

**Date:** 31st January, 2026

**Candidate Details:**
- Name: Sushil Panthi
- Email: npanthi718@gmail.com
- Contact: 9359029905

---

In [22]:
# Import required libraries
import pandas as pd
import json

# Load orders.csv
df_orders = pd.read_csv('orders.csv')
print('Orders Data:')
display(df_orders.head())

# Load users.json
with open('users.json', 'r') as f:
    users_data = json.load(f)
print('Users Data:')
display(pd.DataFrame(users_data).head())

# Preview restaurants.sql (show first 20 lines as text)
print('Restaurants SQL (first 20 lines):')
with open('restaurants.sql', 'r') as f:
    for i, line in enumerate(f):
        if i >= 20:
            break
        print(line.strip())

Orders Data:


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


Users Data:


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


Restaurants SQL (first 20 lines):
CREATE TABLE restaurants (
restaurant_id INT,
restaurant_name TEXT,
cuisine TEXT,
rating FLOAT);

INSERT INTO restaurants VALUES (1, 'Restaurant_1', 'Chinese', 4.8);
INSERT INTO restaurants VALUES (2, 'Restaurant_2', 'Indian', 4.1);
INSERT INTO restaurants VALUES (3, 'Restaurant_3', 'Mexican', 4.3);
INSERT INTO restaurants VALUES (4, 'Restaurant_4', 'Chinese', 4.1);
INSERT INTO restaurants VALUES (5, 'Restaurant_5', 'Chinese', 4.8);
INSERT INTO restaurants VALUES (6, 'Restaurant_6', 'Chinese', 4.8);
INSERT INTO restaurants VALUES (7, 'Restaurant_7', 'Italian', 3.2);
INSERT INTO restaurants VALUES (8, 'Restaurant_8', 'Indian', 4.8);
INSERT INTO restaurants VALUES (9, 'Restaurant_9', 'Italian', 3.1);
INSERT INTO restaurants VALUES (10, 'Restaurant_10', 'Chinese', 3.4);
INSERT INTO restaurants VALUES (11, 'Restaurant_11', 'Chinese', 4.9);
INSERT INTO restaurants VALUES (12, 'Restaurant_12', 'Mexican', 3.4);
INSERT INTO restaurants VALUES (13, 'Restaurant_

In [23]:
# QN. 1 Merge orders and users data to answer the question
import pandas as pd
import json

# Load orders.csv
df_orders = pd.read_csv('orders.csv')

# Load users.json
with open('users.json', 'r') as f:
    users_data = json.load(f)
df_users = pd.DataFrame(users_data)

# Merge on user_id
df_merged = pd.merge(df_orders, df_users, on='user_id', how='left')

# Filter for Gold members
df_gold = df_merged[df_merged['membership'] == 'Gold']

# Group by city and sum total_amount
city_revenue = df_gold.groupby('city')['total_amount'].sum().sort_values(ascending=False)

# Display the city with the highest revenue
print('City with highest total revenue from Gold members:')
display(city_revenue)
print(f"Top city: {city_revenue.idxmax()} with revenue {city_revenue.max():.2f}")

City with highest total revenue from Gold members:


city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

Top city: Chennai with revenue 1080909.79


In [24]:
#QN. 2 To answer this, we need to map each order to its cuisine using restaurant_id from orders.csv and restaurants.sql
import pandas as pd
import re

# Load orders.csv
df_orders = pd.read_csv('orders.csv')

# Parse restaurants.sql to get restaurant_id and cuisine
restaurants = []
with open('restaurants.sql', 'r') as f:
    for line in f:
        match = re.match(r"INSERT INTO restaurants VALUES \((\d+), '.*?', '(.*?)', ([\d.]+)\);", line)
        if match:
            restaurant_id = int(match.group(1))
            cuisine = match.group(2)
            restaurants.append({'restaurant_id': restaurant_id, 'cuisine': cuisine})
df_restaurants = pd.DataFrame(restaurants)

# Merge orders with restaurants on restaurant_id
df_orders = pd.merge(df_orders, df_restaurants, on='restaurant_id', how='left')

# Calculate average order value per cuisine
cuisine_avg = df_orders.groupby('cuisine')['total_amount'].mean().sort_values(ascending=False)

print('Average order value by cuisine:')
display(cuisine_avg)
print(f"Cuisine with highest average order value: {cuisine_avg.idxmax()} ({cuisine_avg.max():.2f})")

Average order value by cuisine:


cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

Cuisine with highest average order value: Mexican (808.02)


In [25]:
#QN. 3  Calculate total order value per user and count users with total > 1000
user_totals = df_orders.groupby('user_id')['total_amount'].sum()
num_users = (user_totals > 1000).sum()
print(f"Number of distinct users with total orders > ₹1000: {num_users}")

Number of distinct users with total orders > ₹1000: 2544


In [26]:
# QN 4 Compute total revenue by restaurant rating ranges and find the top range
import pandas as pd
import re

# Load orders (if not already loaded)
try:
    df_orders
except NameError:
    df_orders = pd.read_csv('orders.csv')

# Parse restaurants.sql to extract restaurant_id and rating
restaurants = []
with open('restaurants.sql', 'r') as f:
    for line in f:
        m = re.match(r"INSERT INTO restaurants VALUES \((\d+), '([^']*)', '([^']*)', ([\d.]+)\);", line)
        if m:
            rid = int(m.group(1))
            name = m.group(2)
            cuisine = m.group(3)
            rating = float(m.group(4))
            restaurants.append({'restaurant_id': rid, 'restaurant_name': name, 'cuisine': cuisine, 'rating': rating})

if not restaurants:
    raise ValueError('No restaurants parsed from restaurants.sql')

df_restaurants = pd.DataFrame(restaurants)

# Merge orders with restaurant ratings
df = pd.merge(df_orders, df_restaurants[['restaurant_id','rating']], on='restaurant_id', how='left')

# Map rating to the provided ranges
def rating_range(r):
    if pd.isna(r):
        return 'Unknown'
    if 3.0 <= r <= 3.5:
        return '3.0 – 3.5'
    if 3.6 <= r <= 4.0:
        return '3.6 – 4.0'
    if 4.1 <= r <= 4.5:
        return '4.1 – 4.5'
    if 4.6 <= r <= 5.0:
        return '4.6 – 5.0'
    return 'Other'

df['rating_range'] = df['rating'].apply(rating_range)

# Sum revenue per range
revenue_by_range = df.groupby('rating_range')['total_amount'].sum().sort_values(ascending=False)

print('Total revenue by rating range:')
display(revenue_by_range)

# Top range
top_range = revenue_by_range.idxmax()
print(f"Rating range with highest total revenue: {top_range} ({revenue_by_range.max():.2f})")

Total revenue by rating range:


rating_range
4.6 – 5.0    2197030.75
3.0 – 3.5    2136772.70
4.1 – 4.5    1960326.26
3.6 – 4.0    1717494.41
Name: total_amount, dtype: float64

Rating range with highest total revenue: 4.6 – 5.0 (2197030.75)


In [27]:
# QN . 5 Compute average order value by city among Gold members
import pandas as pd
import json

# Load orders and users
df_orders = pd.read_csv('orders.csv')
with open('users.json', 'r') as f:
    users = json.load(f)
df_users = pd.DataFrame(users)

# Merge and filter for Gold members
df_merged = pd.merge(df_orders, df_users, on='user_id', how='left')
df_gold = df_merged[df_merged['membership'] == 'Gold']

# Compute average order value per city
city_avg_gold = df_gold.groupby('city')['total_amount'].mean().sort_values(ascending=False)

print('Average order value by city among Gold members:')
display(city_avg_gold)
print(f"Top city among Gold members: {city_avg_gold.idxmax()} ({city_avg_gold.max():.2f})")

Average order value by city among Gold members:


city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

Top city among Gold members: Chennai (808.46)


In [28]:
# QN 6 Compute number of distinct restaurants and total revenue for the selected cuisines
import pandas as pd
import re

# Load orders if not present
try:
    df_orders
except NameError:
    df_orders = pd.read_csv('orders.csv')

# Parse restaurants.sql (reuse df_restaurants if available)
try:
    df_restaurants
except NameError:
    restaurants = []
    with open('restaurants.sql', 'r') as f:
        for line in f:
            m = re.match(r"INSERT INTO restaurants VALUES \((\d+), '([^']*)', '([^']*)', ([\d.]+)\);", line)
            if m:
                rid = int(m.group(1))
                name = m.group(2)
                cuisine = m.group(3)
                rating = float(m.group(4))
                restaurants.append({'restaurant_id': rid, 'restaurant_name': name, 'cuisine': cuisine, 'rating': rating})
    df_restaurants = pd.DataFrame(restaurants)

# Merge orders with restaurants to get cuisine for each order
df = pd.merge(df_orders, df_restaurants[['restaurant_id','cuisine']], on='restaurant_id', how='left')

# Focus on the four cuisines
choices = ['Indian','Chinese','Italian','Mexican']

stats = df.groupby('cuisine').agg(num_restaurants=pd.NamedAgg(column='restaurant_id', aggfunc=lambda x: x.nunique()),
                                      total_revenue=pd.NamedAgg(column='total_amount', aggfunc='sum')).reset_index()

stats_choices = stats[stats['cuisine'].isin(choices)].set_index('cuisine')

# Display results
print('Distinct restaurants and total revenue for selected cuisines:')
display(stats_choices)

# Identify the cuisine with lowest number of distinct restaurants among the choices
min_count = stats_choices['num_restaurants'].min()
lowest_cuisines = stats_choices[stats_choices['num_restaurants'] == min_count]

# Determine whether its revenue is "significant" compared to the median revenue of the four
median_revenue = stats_choices['total_revenue'].median()

for cuisine, row in lowest_cuisines.iterrows():
    is_significant = row['total_revenue'] >= median_revenue
    significance = 'significant' if is_significant else 'not significant'
    print(f"\nCuisine with lowest distinct restaurants: {cuisine} ({int(row['num_restaurants'])} restaurants) — total revenue {row['total_revenue']:.2f}, which is {significance} (median {median_revenue:.2f})")

Distinct restaurants and total revenue for selected cuisines:


,num_restaurants,total_revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09



Cuisine with lowest distinct restaurants: Chinese (120 restaurants) — total revenue 1930504.65, which is not significant (median 1997808.19)


In [29]:
# Recompute percentage of total orders placed by Gold members and print the result
# This cell is placed at the end per your request
import pandas as pd
import json

# Load orders and users (use existing variables if present)
try:
    df_orders
except NameError:
    df_orders = pd.read_csv('orders.csv')

try:
    df_users
except NameError:
    with open('users.json','r') as f:
        users = json.load(f)
    df_users = pd.DataFrame(users)

# Merge and compute
df = pd.merge(df_orders, df_users, on='user_id', how='left')
total_orders = len(df)
gold_orders = df[df['membership']=='Gold'].shape[0]
percentage = round((gold_orders / total_orders) * 100)

print(f"Gold member orders: {gold_orders} / {total_orders} = {percentage}%")

# Print the matching option
options = {40: '40%', 45: '45%', 50: '50%', 55: '55%'}
print('Closest option:', options.get(percentage, f"{percentage}% (not listed)"))

Gold member orders: 4987 / 10000 = 50%
Closest option: 50%


In [30]:
# QN: Which restaurant has the highest average order value but less than 20 total orders?
import pandas as pd

# Load orders
try:
    df_orders
except NameError:
    df_orders = pd.read_csv('orders.csv')

# Compute per-restaurant stats
stats = df_orders.groupby('restaurant_name').agg(total_orders=('order_id','count'), avg_order=('total_amount','mean'))

# Filter restaurants with less than 20 total orders
small_rest = stats[stats['total_orders'] < 20]

# The choices to evaluate
choices = ['Grand Cafe Punjabi','Grand Restaurant South Indian','Ruchi Mess Multicuisine','Ruchi Foods Chinese']

# Get stats for choices that exist in small_rest
choices_stats = small_rest.loc[small_rest.index.intersection(choices)].copy()

print('Restaurants among choices with <20 orders:')
if choices_stats.empty:
    print('None of the choices have fewer than 20 orders. Displaying their stats regardless:')
    choices_stats = stats.loc[stats.index.intersection(choices)].copy()
    display(choices_stats)
else:
    display(choices_stats.sort_values('avg_order', ascending=False))

# Determine top restaurant by avg order value among the applicable set
if not choices_stats.empty:
    top_rest = choices_stats['avg_order'].idxmax()
    top_val = choices_stats['avg_order'].max()
    print(f"Top restaurant: {top_rest} with average order value {top_val:.2f} (and {int(choices_stats.loc[top_rest,'total_orders'])} orders)")
else:
    print('No matching restaurants found.')


Restaurants among choices with <20 orders:


,total_orders,avg_order
restaurant_name,,
Ruchi Foods Chinese,19,686.603158


Top restaurant: Ruchi Foods Chinese with average order value 686.60 (and 19 orders)


In [31]:
# QN: Which combination contributes the highest revenue?
# Options:
# - Gold + Indian cuisine
# - Gold + Italian cuisine
# - Regular + Indian cuisine
# - Regular + Chinese cuisine

import pandas as pd
import json
import re

# Load/orders/users/restaurants if not present
try:
    df_orders
except NameError:
    df_orders = pd.read_csv('orders.csv')

try:
    df_users
except NameError:
    with open('users.json','r') as f:
        df_users = pd.DataFrame(json.load(f))

try:
    df_restaurants
except NameError:
    restaurants = []
    with open('restaurants.sql','r') as f:
        for line in f:
            m = re.match(r"INSERT INTO restaurants VALUES \((\d+), '([^']*)', '([^']*)', ([\d.]+)\);", line)
            if m:
                rid = int(m.group(1))
                name = m.group(2)
                cuisine = m.group(3)
                rating = float(m.group(4))
                restaurants.append({'restaurant_id': rid, 'restaurant_name': name, 'cuisine': cuisine, 'rating': rating})
    df_restaurants = pd.DataFrame(restaurants)

# Merge to get membership and cuisine per order
df = pd.merge(df_orders, df_users[['user_id','membership']], on='user_id', how='left')
df = pd.merge(df, df_restaurants[['restaurant_id','cuisine']], on='restaurant_id', how='left')

# Compute total revenue by membership + cuisine
rev = df.groupby(['membership','cuisine'])['total_amount'].sum().reset_index()

# Define options
options = [
    ('Gold','Indian'),
    ('Gold','Italian'),
    ('Regular','Indian'),
    ('Regular','Chinese')
]

# Extract revenues for the options
option_revs = []
for mem, cuis in options:
    row = rev[(rev['membership']==mem) & (rev['cuisine']==cuis)]
    val = float(row['total_amount'].iloc[0]) if not row.empty else 0.0
    option_revs.append({'membership':mem,'cuisine':cuis,'revenue':val})

df_opts = pd.DataFrame(option_revs).set_index(['membership','cuisine'])
print('Total revenue for each option:')
display(df_opts)

# Find top option
top = df_opts['revenue'].idxmax()
top_rev = df_opts['revenue'].max()
print(f"Top combination: {top[0]} + {top[1]} with revenue {top_rev:.2f}")


Total revenue for each option:


revenue
membership cuisine            
Gold       Indian    979312.31
           Italian  1005779.05
Regular    Indian    992100.27
           Chinese   952790.91

Top combination: Gold + Italian with revenue 1005779.05


In [32]:
# Final cell: compute total revenue by quarter and print the quarter with highest revenue
import pandas as pd

# Ensure orders are loaded
try:
    df_orders
except NameError:
    df_orders = pd.read_csv('orders.csv')

# Parse dates and compute quarters
df_orders['order_date'] = pd.to_datetime(df_orders['order_date'], dayfirst=True, errors='coerce')
df_orders['quarter'] = df_orders['order_date'].dt.quarter

quarter_labels = {1: 'Q1 (Jan–Mar)', 2: 'Q2 (Apr–Jun)', 3: 'Q3 (Jul–Sep)', 4: 'Q4 (Oct–Dec)'}
revenue_q = df_orders.groupby('quarter')['total_amount'].sum().reindex([1,2,3,4]).fillna(0)
revenue_q.index = [quarter_labels[q] for q in revenue_q.index]

print('Total revenue by quarter:')
display(revenue_q)

# Top quarter
top = revenue_q.idxmax()
print(f"Quarter with highest total revenue: {top} ({revenue_q.max():.2f})")

Total revenue by quarter:


Q1 (Jan–Mar)    2010626.64
Q2 (Apr–Jun)    1945348.72
Q3 (Jul–Sep)    2037385.10
Q4 (Oct–Dec)    2018263.66
Name: total_amount, dtype: float64

Quarter with highest total revenue: Q3 (Jul–Sep) (2037385.10)


In [33]:
# Final cell: compute requested numerical answers
import pandas as pd
import json
import re

# Ensure required dataframes are loaded
try:
    df_orders
except NameError:
    df_orders = pd.read_csv('orders.csv')

try:
    df_users
except NameError:
    with open('users.json','r') as f:
        df_users = pd.DataFrame(json.load(f))

# Parse restaurants.sql if df_restaurants missing
try:
    df_restaurants
except NameError:
    restaurants = []
    with open('restaurants.sql','r') as f:
        for line in f:
            m = re.match(r"INSERT INTO restaurants VALUES \((\d+), '([^']*)', '([^']*)', ([\d.]+)\);", line)
            if m:
                rid = int(m.group(1))
                name = m.group(2)
                cuisine = m.group(3)
                rating = float(m.group(4))
                restaurants.append({'restaurant_id': rid, 'restaurant_name': name, 'cuisine': cuisine, 'rating': rating})
    df_restaurants = pd.DataFrame(restaurants)

# Merge orders with users to get membership and city
df_ord_user = pd.merge(df_orders, df_users[['user_id','membership','city']], on='user_id', how='left')

# 1) Total orders placed by Gold members
total_orders_gold = int(df_ord_user[df_ord_user['membership']=='Gold'].shape[0])

# 2) Total revenue from orders placed in Hyderabad city (rounded to nearest integer)
rev_hyderabad = round(df_ord_user[df_ord_user['city']=='Hyderabad']['total_amount'].sum())

# 3) Number of distinct users who placed at least one order
distinct_users_with_orders = int(df_orders['user_id'].nunique())

# 4) Average order value for Gold members (rounded to 2 decimals)
avg_order_gold = round(df_ord_user[df_ord_user['membership']=='Gold']['total_amount'].mean(), 2)

# 5) Number of orders placed for restaurants with rating >= 4.5
# Merge orders with restaurants to get rating
df_ord_rest = pd.merge(df_orders, df_restaurants[['restaurant_id','rating']], on='restaurant_id', how='left')
orders_rating_ge_45 = int(df_ord_rest[df_ord_rest['rating'] >= 4.5].shape[0])

# 6) Number of orders placed in the top revenue city among Gold members only
# Compute revenue per city for Gold members, find top city
gold_by_city = df_ord_user[df_ord_user['membership']=='Gold'].groupby('city')['total_amount'].sum()
if not gold_by_city.empty:
    top_city_gold = gold_by_city.idxmax()
    orders_in_top_city_gold = int(df_ord_user[(df_ord_user['membership']=='Gold') & (df_ord_user['city']==top_city_gold)].shape[0])
else:
    top_city_gold = None
    orders_in_top_city_gold = 0

# Print results
print('Numerical Answers:')
print(f'1) Total orders by Gold members: {total_orders_gold}')
print(f'2) Total revenue from Hyderabad (rounded): {rev_hyderabad}')
print(f'3) Distinct users with at least one order: {distinct_users_with_orders}')
print(f'4) Average order value for Gold members: {avg_order_gold:.2f}')
print(f'5) Orders for restaurants with rating >= 4.5: {orders_rating_ge_45}')
if top_city_gold is not None:
    print(f'6) Orders in top revenue city among Gold members ({top_city_gold}): {orders_in_top_city_gold}')
else:
    print('6) Orders in top revenue city among Gold members: 0')


Numerical Answers:
1) Total orders by Gold members: 4987
2) Total revenue from Hyderabad (rounded): 1889367
3) Distinct users with at least one order: 2883
4) Average order value for Gold members: 797.15
5) Orders for restaurants with rating >= 4.5: 3374
6) Orders in top revenue city among Gold members (Chennai): 1337


In [34]:
# Additional final cell: compute fill-in-the-blank answers programmatically
import pandas as pd
import json
import re
import os

# Load orders
try:
    df_orders
except NameError:
    df_orders = pd.read_csv('orders.csv')

# Load users
try:
    df_users
except NameError:
    with open('users.json','r') as f:
        df_users = pd.DataFrame(json.load(f))

# Load restaurants
try:
    df_restaurants
except NameError:
    restaurants = []
    if os.path.exists('restaurants.sql'):
        with open('restaurants.sql','r') as f:
            for line in f:
                m = re.match(r"INSERT INTO restaurants VALUES \((\d+), '([^']*)', '([^']*)', ([\d.]+)\);", line)
                if m:
                    rid = int(m.group(1))
                    name = m.group(2)
                    cuisine = m.group(3)
                    rating = float(m.group(4))
                    restaurants.append({'restaurant_id': rid, 'restaurant_name': name, 'cuisine': cuisine, 'rating': rating})
    df_restaurants = pd.DataFrame(restaurants)

# 1) Join column between orders and users
join_col = 'user_id' if 'user_id' in df_orders.columns and 'user_id' in df_users.columns else None

# 2) Dataset containing cuisine and rating (file & format)
cuisine_dataset_file = 'restaurants.sql' if os.path.exists('restaurants.sql') else 'restaurants.sql'
cuisine_dataset_format = 'SQL file'

# 3) Total rows in final merged dataset (orders merged with users)
if join_col:
    merged_df = pd.merge(df_orders, df_users, on=join_col, how='left')
else:
    merged_df = df_orders.copy()
final_merged_rows = merged_df.shape[0]

# 4) If no matching user record
missing_behavior = 'NaN (missing values)'

# 5) Pandas function used to combine datasets
pandas_function = 'pd.merge'

# 6) membership originates from
membership_origin = 'users.json'

# 7) Join key for orders -> restaurant details
join_key_rest = 'restaurant_id'

# 8) Column identifying food type
food_type_col = 'cuisine'

# 9) If user places multiple orders, personal details appear
repeat_behavior = 'Repeated for each order (one row per order)'

# Print concise answers
print('Fill-in-the-blank (computed answers):')
print(f"1) Join column: {join_col}")
print(f"2) Cuisine & rating dataset: {cuisine_dataset_file} ({cuisine_dataset_format})")
print(f"3) Total rows in final merged dataset: {final_merged_rows}")
print(f"4) If no matching user record: {missing_behavior}")
print(f"5) Pandas function used: {pandas_function}")
print(f"6) 'membership' originates from: {membership_origin}")
print(f"7) Join key for orders -> restaurant details: {join_key_rest}")
print(f"8) Column identifying food type: {food_type_col}")
print(f"9) If user has multiple orders: {repeat_behavior}")


Fill-in-the-blank (computed answers):
1) Join column: user_id
2) Cuisine & rating dataset: restaurants.sql (SQL file)
3) Total rows in final merged dataset: 10000
4) If no matching user record: NaN (missing values)
5) Pandas function used: pd.merge
6) 'membership' originates from: users.json
7) Join key for orders -> restaurant details: restaurant_id
8) Column identifying food type: cuisine
9) If user has multiple orders: Repeated for each order (one row per order)
